### Census Trade Data

September 30, 2020

Brian Dew

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

In [2]:
# Retrieve data from Census API
d = {}

for flow in ['imports', 'exports']:
    base = f'https://api.census.gov/data/timeseries/intltrade/{flow}/hs'
    params = ['CTY_CODE', 'CTY_NAME', 'ALL_VAL_MO']
    if flow == 'imports':
        params[2] = 'GEN_VAL_MO'
    
    param = ','.join(params)
    url = f'{base}?get={param}&key={census_key}&time=from+2015-01'

    r = requests.get(url).json()

    d[flow] = r

In [3]:
i = d['imports']
df = pd.DataFrame(i[1:], columns=i[0])
df = df[(df['CTY_CODE'].str[0] !='0') & 
        (df['CTY_CODE'].str[1] != 'X')]
df['Name'] = df['CTY_NAME'].str.title()
df['Date'] = pd.to_datetime(df['time'], format='%Y-%m')
imp = (df.rename({'GEN_VAL_MO': 'Imports'}, axis=1)
         .set_index(['Name', 'Date']).Imports.astype(float))

e = d['exports']
df = pd.DataFrame(e[1:], columns=e[0])
df = df[(df['CTY_CODE'].str[0] !='0') & 
        (df['CTY_CODE'].str[1] != 'X')]
df['Name'] = df['CTY_NAME'].str.title()
df['Date'] = pd.to_datetime(df['time'], format='%Y-%m')
exp = (df.rename({'ALL_VAL_MO': 'Exports'}, axis=1)
         .set_index(['Name', 'Date']).Exports.astype(float))

data = ((pd.DataFrame([imp, exp]).T / 1000000)
           .rename({'Korea, South': 'South Korea',
                    'Total For All Countries': 'Total, All Countries'}))

data['Total'] = data['Imports'] + data['Exports']

dt = data.xs('Canada').index[-1]
ltdate = dtxt(dt)['mon1']
index = data.xs(dt, level=1).Total.sort_values(ascending=False).iloc[:26].index

final = data.loc[(index, dt),:].astype(int).droplevel('Date')
dt2 = dt - pd.DateOffset(months = 12)
data2 = data.loc[(index, dt2),:].astype(int).droplevel('Date')
data2.columns = [f'{k} ' for k in final.keys()]
final = final.join(data2).sort_values('Total', ascending=False)
final.index.name = ''

(final.applymap('{:,.0f}'.format)
      .to_csv(data_dir / 'trade_partner.tex', sep='&', 
              line_terminator='\\\ ', quotechar=' '))

write_txt(text_dir / 'trade_date.txt', dtxt(dt)['mon1'])
write_txt(text_dir / 'trade_date2.txt', dtxt(dt2)['mon1'])

shval = (final['Total '].iloc[1:].sum() / 
         final['Total '].iloc[0]) * 100
text = (f'In {ltdate}, trade with the top 25 trading partners (see table) '+
        f'comprises {shval:.1f} percent of total US trade in goods.')
write_txt(text_dir / 'trade_partner_sh.txt', text)
print(text)